In [1]:
import sys
import argparse
import os
import pandas as pd
import datetime as dt
from datetime import timedelta
from datetime import datetime
import numpy as np
import json
import geopandas as gpd
from urllib.request import urlopen

In [2]:
world_cases = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
world_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/2/21,12/3/21,12/4/21,12/5/21,12/6/21,12/7/21,12/8/21,12/9/21,12/10/21,12/11/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7310,7311,7312,7312,7316,7317,7317,7321,7322,7325
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3104,3108,3109,3110,3115,3122,3126,3128,3130,3132
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6083,6089,6096,6103,6111,6114,6122,6126,6132,6137
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,132,132,132,132,133,133,133,133,133,133
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,1735,1735,1735,1735,1735,1735,1735,1736,1737,1737


In [3]:
print(world_cases.columns)

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '12/2/21', '12/3/21', '12/4/21', '12/5/21', '12/6/21', '12/7/21',
       '12/8/21', '12/9/21', '12/10/21', '12/11/21'],
      dtype='object', length=694)


In [4]:
# changing the columns to match the US aggregated data
world_cases = world_cases.drop(['Province/State', 'Lat', 'Long'], axis = 1)

# format the dates
dic = {}
for i in range(len(world_cases.columns)):
    to_replace = list(world_cases.columns)[i].replace('/', '-')
    year = to_replace[-2:]  
    year = "20" + year
    to_replace = to_replace[:len(to_replace) - 3]
    
    to_replace = year + '-' + to_replace
    
    dic[list(world_cases.columns)[i]] = to_replace
    

world_cases = world_cases.rename(index = str, columns = dic)
world_cases = world_cases.rename(columns = {"20on-Country-Reg": "name"})
world_cases.head()

,name,2020-1-22,2020-1-23,2020-1-24,2020-1-25,2020-1-26,2020-1-27,2020-1-28,2020-1-29,2020-1-30,...,2021-12-2,2021-12-3,2021-12-4,2021-12-5,2021-12-6,2021-12-7,2021-12-8,2021-12-9,2021-12-10,2021-12-11
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,7310,7311,7312,7312,7316,7317,7317,7321,7322,7325
1,Albania,0,0,0,0,0,0,0,0,0,...,3104,3108,3109,3110,3115,3122,3126,3128,3130,3132
2,Algeria,0,0,0,0,0,0,0,0,0,...,6083,6089,6096,6103,6111,6114,6122,6126,6132,6137
3,Andorra,0,0,0,0,0,0,0,0,0,...,132,132,132,132,133,133,133,133,133,133
4,Angola,0,0,0,0,0,0,0,0,0,...,1735,1735,1735,1735,1735,1735,1735,1736,1737,1737


In [5]:
# filter out the dates not needed
worldStart = dt.datetime(2020, 1, 22)
startDate = dt.datetime(2020, 2, 14)
endDate = dt.datetime(2021, 12, 9)
# print(int((endDate - startDate).days))

# datetime_columns = pd.to_datetime(world_cases.columns[1:])
# column_drop_list = [i for i in list(world_case.columns) if date]


In [6]:
len(world_cases)

280

In [7]:
# group by name
world_cases = pd.DataFrame(world_cases.groupby(by = ["name"], as_index = False).sum())
world_cases.head()

,name,2020-1-22,2020-1-23,2020-1-24,2020-1-25,2020-1-26,2020-1-27,2020-1-28,2020-1-29,2020-1-30,...,2021-12-2,2021-12-3,2021-12-4,2021-12-5,2021-12-6,2021-12-7,2021-12-8,2021-12-9,2021-12-10,2021-12-11
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,7310,7311,7312,7312,7316,7317,7317,7321,7322,7325
1,Albania,0,0,0,0,0,0,0,0,0,...,3104,3108,3109,3110,3115,3122,3126,3128,3130,3132
2,Algeria,0,0,0,0,0,0,0,0,0,...,6083,6089,6096,6103,6111,6114,6122,6126,6132,6137
3,Andorra,0,0,0,0,0,0,0,0,0,...,132,132,132,132,133,133,133,133,133,133
4,Angola,0,0,0,0,0,0,0,0,0,...,1735,1735,1735,1735,1735,1735,1735,1736,1737,1737


In [8]:
len(world_cases)

196

In [9]:
# convert cumulative to daily
for i in range(len(world_cases.columns) - 1, 2, -1):
    world_cases[world_cases.columns[i]] = world_cases[world_cases.columns[i]].sub(world_cases[world_cases.columns[i - 1]], axis = 0)
# world_cases.head()

In [10]:
world_cases = world_cases.drop(world_cases.columns[1:(1+int((startDate - worldStart).days))], axis = 1)
world_cases = world_cases.drop(world_cases.columns[(1+int((endDate - startDate).days)):], axis = 1)
world_cases.head()

,name,2020-2-14,2020-2-15,2020-2-16,2020-2-17,2020-2-18,2020-2-19,2020-2-20,2020-2-21,2020-2-22,...,2021-11-29,2021-11-30,2021-12-1,2021-12-2,2021-12-3,2021-12-4,2021-12-5,2021-12-6,2021-12-7,2021-12-8
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,1,0,4,1,0
1,Albania,0,0,0,0,0,0,0,0,0,...,3,4,5,3,4,1,1,5,7,4
2,Algeria,0,0,0,0,0,0,0,0,0,...,6,7,5,7,6,7,7,8,3,8
3,Andorra,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,Angola,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0


In [11]:
# merge with the data with geoid
world_geoid = pd.read_csv('data/georegion.csv')
print(world_geoid.columns)
world_cases = world_geoid.merge(world_cases, left_on = world_geoid.columns[0], right_on = "name", how = 'left')


# to_drop = []
# to_drop.append(len(world_cases.columns) - 1)
# for i in range(2, 25):
#     to_drop.append(i)
# world_cases = world_cases.drop(world_cases.columns[to_drop], axis = 1)

world_cases = world_cases.drop(world_cases.columns[2], axis = 1)
world_cases.head()

Index(['﻿geoName', 'geoRegion'], dtype='object')


,﻿geoName,geoRegion,2020-2-14,2020-2-15,2020-2-16,2020-2-17,2020-2-18,2020-2-19,2020-2-20,2020-2-21,...,2021-11-29,2021-11-30,2021-12-1,2021-12-2,2021-12-3,2021-12-4,2021-12-5,2021-12-6,2021-12-7,2021-12-8
0,Antigua and Barbuda,NORTH AMERICA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Algeria,AFRICA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,7.0,5.0,7.0,6.0,7.0,7.0,8.0,3.0,8.0
2,Azerbaijan,ASIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,26.0,23.0,28.0,21.0,24.0,18.0,20.0,16.0,21.0,15.0
3,Albania,EURPOE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,4.0,5.0,3.0,4.0,1.0,1.0,5.0,7.0,4.0
4,Armenia,ASIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21.0,32.0,43.0,21.0,13.0,25.0,14.0,8.0,27.0,10.0


In [12]:
len(world_cases)

238

In [13]:
# world_cases[world_cases['_merge'] != 'both']

In [14]:
len(world_cases)

238

In [15]:
output_cases = pd.read_csv("data/output_deaths.csv")
output_cases.head()


# world_cases
# world_cases.to_csv("data/world_cases.csv", index = False)

,name,state,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,...,2021-11-29,2021-11-30,2021-12-01,2021-12-02,2021-12-03,2021-12-04,2021-12-05,2021-12-06,2021-12-07,2021-12-08
0,Phoenix-Mesa-Scottsdale,AZ,0,0,0,0,0,0,0,0,...,0,48,21,16,5,88,27,0,93,0
1,San Jose-Sunnyvale-Santa Clara,CA,0,0,0,0,0,0,0,0,...,0,5,5,0,3,0,0,5,4,4
2,San Francisco-Oakland-Hayward,CA,0,0,0,0,0,0,0,0,...,2,0,6,6,13,0,0,3,1,2
3,San Diego-Carlsbad,CA,0,0,0,0,0,0,0,0,...,6,10,5,6,4,0,0,1,1,5
4,Los Angeles-Long Beach-Anaheim,CA,0,0,0,0,0,0,0,0,...,45,43,29,37,24,19,2,17,26,26


In [16]:
# world_cases = pd.concat([output_cases, world_cases], axis = 0)
world_cases.columns


Index(['﻿geoName', 'geoRegion', '2020-2-14', '2020-2-15', '2020-2-16',
       '2020-2-17', '2020-2-18', '2020-2-19', '2020-2-20', '2020-2-21',
       ...
       '2021-11-29', '2021-11-30', '2021-12-1', '2021-12-2', '2021-12-3',
       '2021-12-4', '2021-12-5', '2021-12-6', '2021-12-7', '2021-12-8'],
      dtype='object', length=666)

In [17]:
output_cases.columns

Index(['name', 'state', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17',
       '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21',
       ...
       '2021-11-29', '2021-11-30', '2021-12-01', '2021-12-02', '2021-12-03',
       '2021-12-04', '2021-12-05', '2021-12-06', '2021-12-07', '2021-12-08'],
      dtype='object', length=666)

In [18]:
rename_col = {}
for i in range(len(output_cases.columns)):
    rename_col[list(output_cases.columns)[i]] = list(world_cases.columns)[i]
output_cases = output_cases.rename(index = str, columns = rename_col)
# print(rename_col)
# print(output_cases.columns == world_cases.columns)

In [19]:
world_cases = world_cases.append(output_cases, ignore_index = True)

In [20]:
world_cases.head()

,﻿geoName,geoRegion,2020-2-14,2020-2-15,2020-2-16,2020-2-17,2020-2-18,2020-2-19,2020-2-20,2020-2-21,...,2021-11-29,2021-11-30,2021-12-1,2021-12-2,2021-12-3,2021-12-4,2021-12-5,2021-12-6,2021-12-7,2021-12-8
0,Antigua and Barbuda,NORTH AMERICA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Algeria,AFRICA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,7.0,5.0,7.0,6.0,7.0,7.0,8.0,3.0,8.0
2,Azerbaijan,ASIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,26.0,23.0,28.0,21.0,24.0,18.0,20.0,16.0,21.0,15.0
3,Albania,EURPOE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,4.0,5.0,3.0,4.0,1.0,1.0,5.0,7.0,4.0
4,Armenia,ASIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21.0,32.0,43.0,21.0,13.0,25.0,14.0,8.0,27.0,10.0


In [21]:
georegion_af = pd.read_csv("./data/georegion_af.csv")

In [22]:
georegion_af = georegion_af[georegion_af['GEOID'] != -9999]

In [23]:
georegion_af.head()

,GEOID,geoName,geoRegion,geoLON,geoLAT,geoISO3
1,1,Antigua and Barbuda,NORTH AMERICA,-61.791,17.280,ATG
2,2,Algeria,AFRICA,2.632,28.163,DZA
3,3,Azerbaijan,ASIA,47.532,40.292,AZE
4,4,Albania,EURPOE,20.068,41.143,ALB
5,5,Armenia,ASIA,44.948,40.286,ARM


In [24]:
len(georegion_af)

1168

In [25]:
world_cases.columns

Index(['﻿geoName', 'geoRegion', '2020-2-14', '2020-2-15', '2020-2-16',
       '2020-2-17', '2020-2-18', '2020-2-19', '2020-2-20', '2020-2-21',
       ...
       '2021-11-29', '2021-11-30', '2021-12-1', '2021-12-2', '2021-12-3',
       '2021-12-4', '2021-12-5', '2021-12-6', '2021-12-7', '2021-12-8'],
      dtype='object', length=666)

In [26]:
world_cases = world_cases.merge(georegion_af, left_on = [world_cases.columns[0], "geoRegion"] , right_on = ["geoName", "geoRegion"], how = 'right')

In [27]:
# reorder the columns
columns_reordered1 = list(world_cases.columns)[1:2]
columns_reordered2 = list(world_cases.columns)[2:-6]
columns_reordered3 = list(world_cases.columns)[-5:-3]
columns_reordered4 = list(world_cases.columns)[-3:]
columns_reordered = columns_reordered3 + columns_reordered1 + columns_reordered4 + columns_reordered2 


world_cases = world_cases.reindex(columns = columns_reordered)

In [28]:
# change the floats to integers
def float_to_int(x):
    if (x != x):
         # print("hey")
        return -9999
    x = int(x)
    return x

for i in list(world_cases.columns[6:]):
    world_cases[i] = world_cases[i].apply(float_to_int)

def format_float(x):
    x = round(x, 3)
    return x

world_cases['geoLON'] = world_cases['geoLON'].apply(format_float)
world_cases['geoLAT'] = world_cases['geoLAT'].apply(format_float)
    

In [29]:
world_cases.head()

,GEOID,geoName,geoRegion,geoLON,geoLAT,geoISO3,2020-2-14,2020-2-15,2020-2-16,2020-2-17,...,2021-11-28,2021-11-29,2021-11-30,2021-12-1,2021-12-2,2021-12-3,2021-12-4,2021-12-5,2021-12-6,2021-12-7
0,1,Antigua and Barbuda,NORTH AMERICA,-61.791,17.280,ATG,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Algeria,AFRICA,2.632,28.163,DZA,0,0,0,0,...,6,6,7,5,7,6,7,7,8,3
2,3,Azerbaijan,ASIA,47.532,40.292,AZE,0,0,0,0,...,18,26,23,28,21,24,18,20,16,21
3,4,Albania,EURPOE,20.068,41.143,ALB,0,0,0,0,...,4,3,4,5,3,4,1,1,5,7
4,5,Armenia,ASIA,44.948,40.286,ARM,0,0,0,0,...,29,21,32,43,21,13,25,14,8,27


In [30]:
len(world_cases)

1168

In [31]:
world_cases.to_csv("data/world_deaths.csv", index = False)